In [1]:
import numpy as np
import pandas as pd

In [3]:
!pip install openpyxl

In [5]:
# statistical tests TET2

df = pd.read_excel(r"D:\Users\albao\Downloads\TET2_predictions_annotationsall_char.xlsx")


In [28]:
import warnings
import pandas as pd
from scipy import stats

ignore = ['SAMPLE','FILTER','VAR_ID','CHR','REF','ALT','GENE', 'ALLELE','VARIANT_CLASS','BIOTYPE','TYPE','HGVSc','HGVSp','EXON','cDNA_position',
 'CDS_position','Amino_acids','Codons','Protein_position', 'GIVEN_REF','USED_REF','BAM_EDIT','IMPACT','CONSEQUENCE','SOMATIC',
 'PHENO','CLIN_SIG','COSMIC_MATCH','HEURISTIC','PREVIOUSLY_IDENTIFIED','WHITELIST','ARTIFACT','SIFT_DESC','SIFT_SCORE','POLYPHEN_DESC',
 'POLYPHEN_SCORE','CADD_PHRED','CADD_RAW','REF_DEPTH_VIS1','ALT_DEPTH_VIS1','F1R2_VIS1','F2R1_VIS1','REF_DEPTH_VIS3',
 'ALT_DEPTH_VIS3','F1R2_VIS3','F2R1_VIS3','VAF_VIS1','VAF_VIS3','VAF_ABSOLUTA','VAF_RELATIVA','VAF_RATIO','CHIP_driver',
 'FU_yrs','AER','log_positive_AER','log_AER','N','doubling_time','doubling_time_wo_outliers','log_ratio_VAF_years','VAF_absolute_yrs','growth_lm_VAF',
 'growth_lm_log_VAF','fitness_exp','region_normal','region_alt','POS.1',"FEATURE", "STRAND", "IMPACT.1", 'VARIANT_CLASS.1',]
df["CpG variant"] = df["CpG variant"].map({'Not CPG variant':0, 'removed':-1, 'added':1})
warnings.filterwarnings("ignore")

tf = pd.DataFrame()
for item in df.columns:
    if item in ignore:
        continue
    positive = df[df["CHIP_driver"] == "YES"][item]
    negative = df[df["CHIP_driver"] == "NO"][item]
    
    # shapiro wilk drivers
    _, pos_shapiro_p = stats.shapiro(positive)
    _, neg_shapiro_p = stats.shapiro(negative)
    
    # shapiro wilk non drivers
    _, levene_p = stats.levene(positive, negative)
    
    # if normal distribution for both use t test, else use a mannwhitney test
    if pos_shapiro_p > 0.05 and neg_shapiro_p > 0.05 and levene_p > 0.05:
        _, ttest_p = stats.ttest_ind(positive, negative, equal_var=True)
        test_type = "t_test"
        test_p = ttest_p
    else:
        _, mannwhitney_p = stats.mannwhitneyu(positive, negative, alternative='two-sided')
        test_type = "mannwhitney"
        test_p = mannwhitney_p
        tf[item] = [pos_shapiro_p, neg_shapiro_p, levene_p, test_type, test_p]

tf.index = ["shapiro_p_chip", "shapiro_p_nonchip", "levene_p_value", "test_used", "test_p_value"]

tf = tf.transpose()
print(tf)
tf.to_excel("./statistics.xlsx")

                         shapiro_p_chip shapiro_p_nonchip levene_p_value  \
Unnamed: 0                          0.0          0.000003       0.901818   
POS                                 0.0               0.0            0.0   
REFSEQ_MATCH                        NaN               NaN            NaN   
exon                                NaN               NaN            NaN   
Amino_acids.1                       NaN               NaN            NaN   
...                                 ...               ...            ...   
Transcript                          1.0               1.0            NaN   
RegulatoryFeature                   1.0               1.0            NaN   
MotifFeature                        1.0               1.0            NaN   
prediction probabilities            0.0               0.0       0.000064   
binarized prediction                0.0               0.0       0.004259   

                            test_used test_p_value  
Unnamed: 0                mannwhit